# Lab 3 - Genetic Algorithm

## Required dependencies

In [ ]:
%pip install jmetalpy

In [55]:
from scipy.optimize import minimize
import numpy as np
from jmetal.algorithm.singleobjective.genetic_algorithm import GeneticAlgorithm
from jmetal.operator import BinaryTournamentSelection, SBXCrossover, SimpleRandomMutation
from jmetal.util.termination_criterion import StoppingByEvaluations
from jmetal.core.problem import FloatProblem
from jmetal.core.solution import FloatSolution
import plotly.graph_objects as go
from typing import Callable

In [56]:
import logging

logger = logging.getLogger('jmetal.core.algorithm')
logger.setLevel(logging.INFO)

In [57]:
def plot_population_with_contour(
    population: np.ndarray,
    problem: FloatProblem,
    fitness_function: Callable,
    number_of_points: int = 100,
    number_of_contours: int = 20,
) -> None:
    x = np.linspace(problem.lower_bound[0], problem.upper_bound[0], number_of_points)
    y = np.linspace(problem.lower_bound[1], problem.upper_bound[1], number_of_points)
    x_mesh, y_mesh = np.meshgrid(x, y)
    z_mesh = fitness_function(x_mesh, y_mesh)
    fig = go.Figure(
        data=[
            go.Contour(
                z=z_mesh.reshape(-1),
                x=x_mesh.reshape(-1),
                y=y_mesh.reshape(-1),
                colorscale="Viridis",
                contours=dict(
                    start=np.min(z_mesh),
                    end=np.max(z_mesh),
                    size=(np.max(z_mesh) - np.min(z_mesh)) / number_of_contours,
                ),
                colorbar=dict(title="Ackley function value"),
            )
        ]
    )
    fig.add_trace(
        go.Scatter(
            x=population[:, 0],
            y=population[:, 1],
            mode="markers",
            marker=dict(color="red"),
            name="Last Population",
        )
    )

    fig.update_layout(
        title="Contour plot of the Ackley function for the last population",
        showlegend=False,
        width=800,
        height=800,
    )

    fig.show()

### Ackley function

![Ackley](https://www.sfu.ca/~ssurjano/ackley.png)

$$ f(\mathbf{x}) = -a \exp\left(-b \sqrt{\frac{1}{d} \sum_{i=1}^{d} x_i^2}\right) - \exp\left(\frac{1}{d} \sum_{i=1}^{d} \cos(c x_i)\right) + a + \exp(1) $$

In [58]:
def ackley(solution: np.ndarray) -> float:
    a = 20
    b = 0.2
    c = 2 * np.pi
    d = solution.shape[0]
    return (
        -a * np.exp(-b * np.sqrt(1 / d * np.sum(np.power(solution, 2))))
        - np.exp(1 / d * np.sum(np.cos(c * solution)))
        + a
        + np.e
    )


def ackley_2d(x: np.ndarray, y: np.ndarray) -> list[float]:
    values = [ackley(np.array([x_, y_])) for x_, y_ in zip(x.flatten(), y.flatten())]
    return np.asarray(values).reshape(x.shape)

Your task is to finalize the implementation of the Ackley function, utilizing the function that is defined above.

In [11]:
class Ackley(FloatProblem):
    def __init__(self, number_of_variables: int):
        super(Ackley, self).__init__()
        number_of_variables = number_of_variables

        self.obj_directions = [self.MINIMIZE]
        self.obj_labels = ["f(x)"]

        self.lower_bound = [-32.768 for _ in range(number_of_variables)]
        self.upper_bound = [32.768 for _ in range(number_of_variables)]

        FloatSolution.lower_bound = self.lower_bound
        FloatSolution.upper_bound = self.upper_bound

    def number_of_objectives(self) -> int:
        return 1

    def number_of_constraints(self) -> int:
        return 0

    # TODO: Implement this method
    # Please refer to the https://github.com/jMetal/jMetalPy/blob/main/jmetal/problem/singleobjective/unconstrained.py for further guidance.
    def evaluate(self, solution: FloatSolution) -> FloatSolution:
        raise NotImplementedError()

    def name(self) -> str:
        return "Ackley"

Run genetic algorithm for Ackley 2D.

In [ ]:
d = 2
problem = Ackley(d)

MAX_EVALUATIONS = 1000

algorithm = GeneticAlgorithm(
    problem=problem,
    population_size=50,
    offspring_population_size=100,
    mutation=SimpleRandomMutation(0.1),
    crossover=SBXCrossover(0.95),
    selection=BinaryTournamentSelection(),
    termination_criterion=StoppingByEvaluations(max_evaluations=MAX_EVALUATIONS)
)

algorithm.run()
result = algorithm.result()

print('Algorithm: {}'.format(algorithm.name))
print('Problem: {}'.format(problem.name()))
print('Solution: {}'.format(result.variables))
print('Fitness: {}'.format(result.objectives[0]))
print('Computing time: {}'.format(algorithm.total_computing_time))

In [ ]:
population = np.asarray([solution.variables for solution in algorithm.solutions])

plot_population_with_contour(
    population=population,
    problem=problem,
    fitness_function=ackley_2d,
    number_of_points=100,
    number_of_contours=20,
)

### Multi-start

Multi-start is a technique used in optimization to improve the chances of finding the global minimum of a function. The idea behind multi-start is to run a local optimization algorithm multiple times, each time starting from a different initial point in the search space. By doing this, we increase the chances of finding the global minimum, as each run of the local optimization algorithm may converge to a different local minimum.

The multi-start method works as follows:

1. Select a random starting point (e.g. with uniform distribution over domain).
2. For selected starting point, run the local optimization algorithm until convergence.
3. Record the best solution found across all runs of the local optimization algorithm.
4. Repeat 1-3 `N` times (or repeat until the budget is reached).

We use [L-BFGS-B](https://docs.scipy.org/doc/scipy/reference/optimize.minimize-lbfgsb.html) as a local method.

Our objective is to conduct a comparative analysis between a multi-start method and a genetic algorithm, with the aim to demonstrate that genetic algorithms can offer superior efficiency for certain problem types.

In [50]:
ackley_bounds = [(-32.768, 32.768)] * d

`scipy.optimize.minimize` requires slightly different parameters than `jMetalPy`.

In [ ]:
x0 = [1,2]
res = minimize(ackley, x0, method='L-BFGS-B', bounds=ackley_bounds)
res

Complete the implementation - generate random starting point from the domain.

In [52]:
def multi_start_optimization(func, bounds: list[tuple[float, float]], max_evaluations: int, max_iter: int = 100):
    """
    A multi-start optimization algorithm using scipy.optimize.minimize with the L-BFGS-B method.

    :param func: The objective function to minimize.
    :param bounds: Bounds for variables as a list of tuples.
    :param n_starts: Number of starting points for the optimization.
    :return: A list of results from each optimization start.
    """
    results = []
    number_of_evaluations = 0
    while number_of_evaluations < max_evaluations:
        # Generate a random starting point within the bounds.
        # Use uniform distribution. 
        raise NotImplementedError()
        x0 = ...

        # Perform the optimization
        # Run L-BGFBS-B for a predefined number of iterations (max_iter).
        result = ...
        number_of_evaluations += result.nfev
        results.append(result)

    best_result = min(results, key=lambda result: result.fun)
    return best_result.x, best_result.fun, number_of_evaluations

In [ ]:
x, fitness_value, number_of_evaluations = multi_start_optimization(
    ackley, ackley_bounds, MAX_EVALUATIONS
)
print(f"Best solution: {x}")
print(f"Fitness value: {fitness_value}")
print(f"Number of evaluations: {number_of_evaluations}")

Exercise 1.

Compare genetic algorithm with multi start. For different values of `MAX_EVALUATIONS` (e.g. 100, 500, 1000, 5000, 10000) run both methods 20 times and use boxplots to compare results (you can create boxplots for best fitness values).

Exercise 2.

Implement a hybrid optimization technique coupling an evolutionary and a local search algorithm. Run genetic algorithm with adjusted `MAX_EVALUATIONS` (e.g. 9900 instead of 10000) and then run local method (L-BFGS) for the best solution. 

Exercise 3.

Implement a normal mutation.

$$ x' = x + \mathcal{N}(0, \sigma) $$
$$ \sigma = \frac{upper - lower}{6} $$

Please make sure that $x' \in [lower, upper]$. Run genetic algorithm using normal mutation.

Example of custom mutation:

In [54]:
import random
from jmetal.core.operator import Mutation
from jmetal.core.solution import FloatSolution

class StrangeMutation(Mutation[FloatSolution]):

    def __init__(self, probability: float):
        super(StrangeMutation, self).__init__(probability=probability)

    def execute(self, solution: FloatSolution) -> FloatSolution:
        assert type(solution) is FloatSolution

        for i in range(solution.number_of_variables):
            rand = random.random()
            if rand <= self.probability:
                solution.variables[i] = solution.variables[i] * random.random()
        return solution

    def get_name(self):
        return 'Strange random mutation'

Exercise 4.

You can use any methods you know. Try to obtain the best fitness value for Ackley 10D.